In [ ]:
#default_exp docindex

# docindex
> Generate an index related to all entities rendered with `ShowDoc`

In [8]:
#export
from nbdev.export import nbglob, get_config
from functools import partial
from fastcore.utils import Path
from fastcore.test import test_eq
import re

_re_name = re.compile(r'<DocSection type="\S+" name="(\S+)"')
_re_slug = re.compile(r'---.*slug: (\S+).*---', flags=re.DOTALL)

In [9]:
#hide
from nbdoc.showdoc import ShowDoc
import test_lib.example as ex

In [10]:
#hide
p = Path('test_files/_md_files/test_docs.md')
p.write_text(ShowDoc(ex.function_with_types_in_docstring).jsx)

frontmatter = """---
key2: value2
slug: custom/pathfor/site
key: value
---
"""
p = Path('test_files/_md_files/front_matter_test_docs.md')
p.write_text(frontmatter + ShowDoc(ex.function_with_pep484_type_annotations).jsx);

In [11]:
#export
mdglob = partial(nbglob, recursive=True, extension='.md', config_key='doc_path')

def build_index(path=None):
    "Build an index of names generated with `ShowDoc` to document paths."
    cfg = get_config()
    if path: 
        path = Path(path)
    else:
        path = cfg['doc_path']
    
    doc_host = cfg['doc_host']
    base_url = cfg['doc_baseurl']
    
    if doc_host.endswith('/'): doc_host = doc_host[:-1]
    if not base_url.startswith('/'): base_url = '/' + base_url
    if not base_url.endswith('/'): base_url += '/'
    doc_url = doc_host + base_url
    
    files = mdglob(path) if path else mdglob(config_key='doc_path')
    reverse_idx = {}
    for f in files:
        txt = f.read_text()
        names = _re_name.findall(txt)
        slug_match = _re_slug.search(txt)
        
        if slug_match: 
            doc_path = slug_match.group(1)
        else:
            doc_path = str(f.relative_to(path).with_suffix(''))

        for n in names: reverse_idx[n] = doc_url + doc_path
    return reverse_idx

`build_index` will build an index to names generated with `ShowDoc` to document paths that we can later use to construct links for documentation.

Consider the follwing two markdown files, `test_docs.md` and `front_matter_with_test_docs.md`:

In [12]:
_p1 = Path('test_files/_md_files/test_docs.md')
print(_p1.read_text())

<DocSection type="function" name="function_with_types_in_docstring" module="test_lib.example" heading_level="3">
<SigArgSection>
<SigArg name="param1" /><SigArg name="param2" />
</SigArgSection>
<Description summary="Example function with types documented in the docstring." extended_summary="`PEP 484`_ type annotations are supported. If attribute, parameter, and\nreturn types are annotated according to `PEP 484`_, they do not need to be\nincluded in the docstring:" />
<ParamSection name="Parameters">
	<Parameter name="param1" type="int" desc="The first parameter. something something\nsecond line. foo" />
	<Parameter name="param2" type="str" desc="The second parameter." />
</ParamSection>
<ParamSection name="Returns">
	<Parameter type="bool" desc="True if successful, False otherwise." />
</ParamSection>
</DocSection>


In [13]:
_p2 = Path('test_files/_md_files/front_matter_test_docs.md')
print(_p2.read_text())

---
key2: value2
slug: custom/pathfor/site
key: value
---
<DocSection type="function" name="function_with_pep484_type_annotations" module="test_lib.example" heading_level="3">
<SigArgSection>
<SigArg name="param1" type="int" /><SigArg name="param2" type="str" />
</SigArgSection>
<Description summary="Example function with PEP 484 type annotations." extended_summary="The return type must be duplicated in the docstring to comply\nwith the NumPy docstring style." />
<ParamSection name="Parameters">
	<Parameter name="param1" desc="The first parameter." />
	<Parameter name="param2" desc="The second parameter." />
</ParamSection>
<ParamSection name="Returns">
	<Parameter type="bool" desc="True if successful, False otherwise." />
</ParamSection>
</DocSection>


Notice that for `front_matter_test_docs.md`, the front matter has a `slug`, which is used for the path rather than the directory in which the document resides.

In [14]:
test_eq(build_index('test_files/'), {'function_with_pep484_type_annotations': 'https://outerbounds.github.io/nbdoc/custom/pathfor/site',
                                     'function_with_types_in_docstring': 'https://outerbounds.github.io/nbdoc/_md_files/test_docs'})